In [0]:
'''
Definindo funções da camada Gold a serem utilizadas entre notebooks
'''
from pyspark.sql import functions as F
import datetime as dt

# Funções de Teste

def assert_dataframes(df1, col1, df2, col2):
    print(f"Comparando a coluna {col1} do DF1 com {col2} do DF")
    # Cria um DataFrame com os valores distintos da coluna do segundo DataFrame
    df2_values = df2.select(col2).distinct()
    
    # Adiciona uma coluna indicando se o valor da coluna do primeiro DataFrame está na lista de valores do segundo DataFrame
    comparison = df1.withColumn(
        "is_in_second_dataframe",
        F.when(
            F.col(col1).isNull() | F.col(col1).isin([row[col2] for row in df2_values.collect()]), 
            F.lit(True)
        ).otherwise(F.lit(False))
    )
    
    # Conta o número de registros onde a condição não é atendida
    invalid_count = comparison.filter(F.col("is_in_second_dataframe") == F.lit(False)).count()
    
    # Lança uma exceção se houver registros inválidos
    assert invalid_count == 0, f"Existem {invalid_count} registros em '{col1}' que não estão presentes em '{col2}' e não são NULL"


def test_col_not_null(df, col_name):
    print(f"Avaliando a condição {col_name} não contem nulos")
    # Filtra linhas onde a coluna é nula
    null_count = df.filter(F.col(col_name).isNull()).count()
    
    # Verifica se o número de valores nulos é zero
    assert null_count == 0, f"Coluna {col_name} contém valores null"

def test_biggest_date_before_current_date(df, col_name):
    print(f"Avaliando a condição {col_name} menor que a data atual")
    # Obter a data atual
    current_date = F.current_date()

    # Obter a maior data na coluna especificada
    max_date = df.agg(F.max(col_name)).collect()[0][0]
    
    if isinstance(max_date, str):
        max_date = dt.datetime.strptime(max_date,'%Y%m%d').date()
    
    # Verificar se a maior data é antes da data atual
    assert max_date <= df.select(current_date).collect()[0][0], f"A maior data na coluna {col_name} é posterior a data atual"


def test_value_range(df, col_name, condition):
    df_filtered = df.filter(F.expr(f"{col_name} {condition}"))
    print(f"Avaliando a condição {col_name} {condition}")
    #df_filtered.display()
    assert df.count() == df_filtered.count(), f"Os dados da coluna {col_name} não atendem a condição {condition}"


def test_weekday(df, col_name):
    print(f"Avaliando a condição {col_name} está dentre os valores aceitos para o dia de semana")
    # Obter os valores distintos da coluna
    distinct_values = df.select(col_name).distinct().collect()
    
    # Converter os valores distintos para uma lista de strings
    distinct_values_list = [row[col_name] for row in distinct_values]
    
    # Lista de dias da semana
    weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    
    # Verificar se todos os valores distintos estão na lista de dias da semana
    for value in distinct_values_list:
        assert value in weekdays, f"Os dados da coluna {col_name} não correspondem a dias de semana: {value}"

def test_IATA_codes(df, col_name):
    print(f"Avaliando a condição {col_name} contem 3 caracteres")
    nb_different = df.filter(F.length(F.col(col_name)) != 3).count()
    assert nb_different == 0, "Existem códigos IATA além de 3 dígitos"

def test_ICAO_codes(df, col_name):
    print(f"Avaliando a condição {col_name} contem 4 caracteres")
    nb_different = df.filter(F.length(F.col(col_name)) != 4).count()
    assert nb_different == 0, "Existem códigos IATA além de 4 dígitos"